In [1]:
!pip install icecream

In [2]:
import numpy as np
#import pandas as pd
#import math
#import scipy
from fractions import Fraction
#from icecream import ic

# Doomsday Fuel
=============

Making fuel for the LAMBCHOP's reactor core is a tricky process because of the exotic matter involved. It starts as raw ore, then during processing, begins randomly changing between forms, eventually reaching a stable form. There may be multiple stable forms that a sample could ultimately reach, not all of which are useful as fuel. 

Commander Lambda has tasked you to help the scientists increase fuel creation efficiency by predicting the end state of a given ore sample. You have carefully studied the different structures that the ore can take and which transitions it undergoes. It appears that, while random, the probability of each structure transforming is fixed. That is, each time the ore is in 1 state, it has the same probabilities of entering the next state (which might be the same state).  You have recorded the observed transitions in a matrix. The others in the lab have hypothesized more exotic forms that the ore can become, but you haven't seen all of them.

Write a function solution(m) that takes an array of array of nonnegative ints representing how many times that state has gone to the next state and return an array of ints for each terminal state giving the exact probabilities of each terminal state, represented as the numerator for each state, then the denominator for all of them at the end and in simplest form. The matrix is at most 10 by 10. It is guaranteed that no matter which state the ore is in, there is a path from that state to a terminal state. That is, the processing will always eventually end in a stable state. The ore starts in state 0. The denominator will fit within a signed 32-bit integer during the calculation, as long as the fraction is simplified regularly. 

For example, consider the matrix m:
~~~
[
  [0,1,0,0,0,1],  # s0, the initial state, goes to s1 and s5 with equal probability
  [4,0,0,3,2,0],  # s1 can become s0, s3, or s4, but with different probabilities
  [0,0,0,0,0,0],  # s2 is terminal, and unreachable (never observed in practice)
  [0,0,0,0,0,0],  # s3 is terminal
  [0,0,0,0,0,0],  # s4 is terminal
  [0,0,0,0,0,0],  # s5 is terminal
]
~~~
So, we can consider different paths to terminal states, such as:
~~~
s0 -> s1 -> s3
s0 -> s1 -> s0 -> s1 -> s0 -> s1 -> s4
s0 -> s1 -> s0 -> s5
~~~
Tracing the probabilities of each, we find that
s2 has probability 0
s3 has probability 3/14
s4 has probability 1/7
s5 has probability 9/14
So, putting that together, and making a common denominator, gives an answer in the form of
`[s2.numerator, s3.numerator, s4.numerator, s5.numerator, denominator]` which is
`[0, 3, 2, 9, 14]`.

Languages
=========

To provide a Java solution, edit Solution.java
To provide a Python solution, edit solution.py

Test cases
==========
Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.

-- Java cases --
Input:
Solution.solution({{0, 2, 1, 0, 0}, {0, 0, 0, 3, 4}, {0, 0, 0, 0, 0}, {0, 0, 0, 0,0}, {0, 0, 0, 0, 0}})
Output:
    [7, 6, 8, 21]

Input:
Solution.solution({{0, 1, 0, 0, 0, 1}, {4, 0, 0, 3, 2, 0}, {0, 0, 0, 0, 0, 0}, {0, 0, 0, 0, 0, 0}, {0, 0, 0, 0, 0, 0}, {0, 0, 0, 0, 0, 0}})
Output:
    [0, 3, 2, 9, 14]

-- Python cases --
Input:
solution.solution([[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0,0], [0, 0, 0, 0, 0]])
Output:
    [7, 6, 8, 21]

Input:
solution.solution([[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
Output:
    [0, 3, 2, 9, 14]

Use verify [file] to test your solution and see how it does. When you are finished editing your code, use submit [file] to submit your answer. If your solution passes the test cases, it will be removed from your home folder.

In [3]:
def solution(m):
    #m = np.array(m)
    n = len(m)
    m = pd.DataFrame(m)

    #standard form
    idx_terminal = []
    #row_sum = [] #divisor for standard form
    for idx, row in m.iterrows():
        if row.sum() == 0:
            idx_terminal.append(idx)
            row[idx] = 1
        #row_sum.append(sum(row))
    idx_nonterminal = [i for i in range(n) if i not in idx_terminal]

    m = m.reindex(idx_terminal + idx_nonterminal, columns = idx_terminal + idx_nonterminal)
    #m = m/m['row_sum']
    m = m.div(m.sum(axis=1), axis=0) #.iloc[0:n, 0:n]
    r = np.array(m.loc[idx_nonterminal, idx_terminal])
    q = np.array(m.loc[idx_nonterminal, idx_nonterminal], dtype=Fraction)#.astype(Fraction)
    
    f = np.linalg.inv(np.eye(len(idx_nonterminal), dtype=Fraction) - q)

    fr = f.dot(r)
    fr = fr.sum(axis=0)

    denominators = [Fraction(r).denominator for r in fr]
    print(denominators)
    lcm = denominators[0]
    for d in denominators[1:]:
        lcm = lcm // math.gcd(lcm, d) * d
    return lcm

    print(idx_nonterminal, idx_terminal)
    print(fr.astype(Fraction))
    #return m[max(idx_terminal):n, 0:idx_terminal]



In [4]:
def find_terminal_nonterminal_states(m):
    n = len(m)
    idx_term = []
    #row_sum = [] #divisor for standard form
    for i in range(n):
        if m[i].sum() == 0:
            idx_term.append(i)
    idx_nonterm = [i for i in range(n) if i not in idx_term]
    
    return idx_term, idx_nonterm

def get_r_q_matrices(m, idx_nonterm, idx_term):
    m = m[idx_nonterm]
    ''' max_of_nonterminal_states = np.max(m[idx_nonterm], axis=1)
    for i in range(len(r)):
        r[i] = r[i] / max_of_nonterminal_states[i]'''
    sum_of_nonterminal_states = np.sum(m, axis=1)
    #print(max_of_nonterminal_states[0])
    for i in range(len(m)):
        m[i] = m[i] / Fraction(sum_of_nonterminal_states[i])
    r = m[:, idx_term]
    q = m[:, idx_nonterm]
    
    return r, q

# don't reinvent the wheel, inverse of a matrix
# https://www.delftstack.com/howto/python/inverse-of-matrix-in-python/
def return_transpose(mat):
    return map(list,zip(*mat))

def return_matrix_minor(mat,i,j):
    return [row[:j] + row[j+1:] for row in (mat[:i]+mat[i+1:])]

def return_determinant(mat):
    if len(mat) == 2:
        return mat[0][0]*mat[1][1]-mat[0][1]*mat[1][0]

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1)**c)*m[0][c]*return_determinant(return_matrix_minor(m,0,c))
    return determinant

def inverse_matrix(m):
    determinant = return_determinant(m)
    if len(m) == 2:
        return [[m[1][1]/determinant, -1*m[0][1]/determinant],
                [-1*m[1][0]/determinant, m[0][0]/determinant]]

    cfs = []
    for r in range(len(m)):
        cfRow = []
        for c in range(len(m)):
            minor = return_matrix_minor(m,r,c)
            cfRow.append(((-1)**(r+c)) * return_determinant(minor))
        cfs.append(cfRow)
    cfs = return_transpose(cfs)
    for r in range(len(cfs)):
        for c in range(len(cfs)):
            cfs[r][c] = cfs[r][c]/determinant
    return cfs
#
 
        
def calc_f_matrix(q):
    i = np.eye(len(q), dtype=Fraction)
    f = np.array(inverse_matrix(i - q))
    
    return f
    
def solution(m):
    m = np.array(m, dtype=Fraction)
    
    #
    idx_term, idx_nonterm = find_terminal_nonterminal_states(m)
    
    r, q = get_r_q_matrices(m, idx_nonterm, idx_term)
    
    f = calc_f_matrix(q)
    
    fr = f.dot(r)
    
    denominator = np.lcm.reduce([i.denominator for i in fr[0]])
    res = [int(i.numerator * denominator / i.denominator) for i in fr[0]]
    res.append(denominator)
    
    #print(m, idx_term, idx_nonterm)
    #print(q, "\n", "\n", f)
    print(res)#, fr[0])

In [5]:
print(solution([[0, 1, 0, 0, 0, 1],
                [4, 0, 0, 3, 2, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0]])) 

[0, 3, 2, 9, 14]
None


In [6]:
print(solution([[0, 2, 1, 0, 0],
                [0, 0, 0, 3, 4],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0]])) # [7, 6, 8, 21]

[7, 6, 8, 21]
None


In [7]:
np.eye(2, dtype=Fraction)#[0][0].numerator

array([[1, 0],
       [0, 1]], dtype=object)

In [8]:
m = np.array([[0, 2, 1, 0, 0],
                [0, 0, 0, 3, 4],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0]], dtype=Fraction)
idx_term = [2, 3, 4]
idx_nonterm = [0, 1]
#print(m[idx_nonterm])
#print(np.max(m[idx_nonterm], axis=1))
#print(m[idx_term])
r = m[idx_nonterm]
#print(r[0])
sum_of_nonterminal_states = np.sum(m[idx_nonterm], axis=1)
#print(sum_of_nonterminal_states)
for i in range(len(r)):
    r[i] = r[i] / Fraction(sum_of_nonterminal_states[i])
print(r)
print(r[:,idx_term])
#print(type(m[0][1]))

[[Fraction(0, 1) Fraction(2, 3) Fraction(1, 3) Fraction(0, 1)
  Fraction(0, 1)]
 [Fraction(0, 1) Fraction(0, 1) Fraction(0, 1) Fraction(3, 7)
  Fraction(4, 7)]]
[[Fraction(1, 3) Fraction(0, 1) Fraction(0, 1)]
 [Fraction(0, 1) Fraction(3, 7) Fraction(4, 7)]]


In [9]:
print(solution([[0, 2, 1, 0, 0],
                [0, 0, 0, 3, 4],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0]])) # [7, 6, 8, 21]
print(solution([[0, 1, 0, 0, 0, 1],
                [4, 0, 0, 3, 2, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0]])) # [0, 3, 2, 9, 14]


[7, 6, 8, 21]
None
[0, 3, 2, 9, 14]
None


In [10]:
solution([[0, 2, 1, 0, 0],
                [0, 0, 0, 3, 4],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0]])

[7, 6, 8, 21]


In [11]:
solution([[0, 1, 0, 0, 0, 1],
                [4, 0, 0, 3, 2, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0]])

[0, 3, 2, 9, 14]


In [14]:


from typing import List
from math import gcd
from fractions import Fraction

"""
example matrix:
[
    [0, 1, 0, 0, 0, 1],
    [4, 0, 0, 3, 2, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0]
]
"""

def transform_matrix(m: List[List]):
    """
    example matrix becomes
    [
        [0, 1/2, 0, 0, 0, 1/2],
        [4/9, 0, 0, 1/3, 2/9, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]
    ]
    """
    l = len(m)
    for i in range(l):
        row_sum = sum(m[i])
        if row_sum == 0:
            m[i][i] = 1
        else:
            for j in range(l):
                m[i][j] = Fraction(m[i][j], row_sum)

def get_submatrix(matrix: List[List], rows: List, cols: List) -> List[List]:
    new_matrix = []
    for row in rows:
        current_row = []
        for col in cols:
            current_row.append(matrix[row][col])
        new_matrix.append(current_row)
    return new_matrix

def get_q_matrix(matrix: List[List], transient_states: List) -> List[List]:
    """
    For example matrix
    q_matrix = [
        [0, 1/2],
        [4/9, 0]
    ]
    """
    return get_submatrix(matrix, transient_states, transient_states)

def get_r_matrix(matrix: List[List], transient_states: List, absorbing_states: List) -> List[List]:
    """
    For example matrix
    r_matrix = [
        [0, 0, 0, 1/2],
        [0, 1/3, 2/9, 0]
    ]
    """
    return get_submatrix(matrix, transient_states, absorbing_states)

def make_2d_list(n: int, m: int) -> List[List]:
    a = []
    for row in range(n):
        a += [[0]*m]
    return a

def make_identity(n: int) -> List[List]:
    """
    for n=3, it returns
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ]
    """
    matrix = make_2d_list(n, n)
    for i in range(n):
        matrix[i][i] = 1
    return matrix

def subtract_matrices(a: List[List], b: List[List]) -> List[List]:
    new_matrix = []
    n, m = len(a), len(b)
    for i in range(n):
        row = []
        for j in range(m):
            row.append(a[i][j] - b[i][j])
        new_matrix.append(row)
    return new_matrix

def multiply_matrices(a: List[List], b: List[List]) -> List[List]:
    """
    Multiply two matrices a and b
    matrix a of size A X B and matrix b of size B X C
    would yield a matrix c of size A X C
    """
    ar, ac, bc = len(a), len(a[0]), len(b[0])
    c = make_2d_list(ar, bc)
    for i in range(ar):
        for j in range(bc):
            prod = Fraction(0, 1)
            for k in range(ac):
                prod += a[i][k] * b[k][j]
            c[i][j] = prod
    return c

def multiply_row_of_square_matrix(matrix: List[List], row: int, k: int) -> List[List]:
    n = len(matrix)
    identity = make_identity(n)
    identity[row][row] = k
    return multiply_matrices(identity, matrix)

def add_multiple_of_row_of_square_matrix(matrix: List[List], source_row: int, k: int, target_row: int):
    """
    add k * source_row to target_row of matrix m
    """
    n = len(matrix)
    row_operator = make_identity(n)
    row_operator[target_row][source_row] = k
    return multiply_matrices(row_operator, matrix)

def invert_matrix(matrix: List[List]) -> List[List]:
    n = len(matrix)
    inverse = make_identity(n)
    for col in range(n):
        diagonal_row = col
        k = Fraction(1, matrix[diagonal_row][col])
        matrix = multiply_row_of_square_matrix(matrix, diagonal_row, k)
        inverse = multiply_row_of_square_matrix(inverse, diagonal_row, k)
        source_row = diagonal_row
        for target_row in range(n):
            if source_row != target_row:
                k = -matrix[target_row][col]
                matrix = add_multiple_of_row_of_square_matrix(matrix, source_row, k, target_row)
                inverse = add_multiple_of_row_of_square_matrix(inverse, source_row, k, target_row)
    return inverse

def lcm(a: int, b: int) -> int:
    result = a * b // gcd(a, b)
    return result

def lcm_for_arrays(args: List) -> int:
    array_length = len(args)
    if array_length <= 2:
        return lcm(*args)

    initial = lcm(args[0], args[1])
    i = 2
    while i < array_length:
        initial = lcm(initial, args[i])
        i += 1
    return initial

def solution(m):
    """
    For example matrix
    transient_states = [0, 1]
    absorbing_states = [2, 3, 4, 5]
    """
    transient_states = []
    absorbing_states = []
    for i in range(len(m)):
        row = m[i]
        if sum(row) == 0:
            absorbing_states.append(i)
        else:
            transient_states.append(i)

    transform_matrix(m)
    
    print(m, '\n')
    
    q = get_q_matrix(m, transient_states)
    r = get_r_matrix(m, transient_states, absorbing_states)
    identity = make_identity(len(q))
    diff = subtract_matrices(identity, q)
    inverse = invert_matrix(diff)
    result = multiply_matrices(inverse, r)
    print('initial result', result)

    denominator = lcm_for_arrays([item.denominator for item in result[0]])
    result = [item.numerator * denominator // item.denominator for item in result[0]]
    result.append(denominator)
    return result

if __name__ == "__main__":
    m1 = [
        [0, 1, 0, 0, 0, 1],
        [4, 0, 0, 3, 2, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]
    ]
    print(solution(m1))


[[Fraction(0, 1), Fraction(1, 2), Fraction(0, 1), Fraction(0, 1), Fraction(0, 1), Fraction(1, 2)], [Fraction(4, 9), Fraction(0, 1), Fraction(0, 1), Fraction(1, 3), Fraction(2, 9), Fraction(0, 1)], [0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]] 

initial result [[Fraction(0, 1), Fraction(3, 14), Fraction(1, 7), Fraction(9, 14)], [Fraction(0, 1), Fraction(3, 7), Fraction(2, 7), Fraction(2, 7)]]
[0, 3, 2, 9, 14]
